In [42]:
import numpy as np, pandas as pd
import seaborn as sns
from sklearn.base import BaseEstimator, TransformerMixin
import cython

%matplotlib inline
%reload_ext Cython

In [43]:
X_train, y_train, X_test = [pd.read_csv(csv)
                            for csv in ['X_train.csv', 'y_train.csv', 'X_test.csv']]

In [44]:
m = X_train.as_matrix()

In [46]:
def feat_indices(X, feats):
    return np.array([X.columns.get_loc(f) for f in feats])

def cat_feats_indices(X, add_cat_feats=[]):
    return np.where(np.array([(X[c].dtype == object)
                                  or (c.endswith('code'))
                                  or (c in add_cat_feats) for c in X.columns.values]))[0]

In [47]:
class Catenc(BaseEstimator, TransformerMixin):
    def __init__(self, cat_feats):
        self.cat_feats = cat_feats
        self.labels_ = None
    def fit(self, X, y=None):
        findices = feat_indices(X, cat_feats)
        X = X.values
        self.labels_ = _catenc_fit(X, findices)
    def transform(self, X):
        findices = feat_indices(X, cat_feats)
        X = X.values
        _catenc_transform(X, findices, self.labels_)